#### Codigo Pasantía

In [2]:
import pandas as pd
import numpy as np
import requests
import os
import re

In [12]:
from tqdm.auto import tqdm

c:\Users\Cristobal\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
EXCEL_FILE = 'ordenanzas.xlsx' 
# 2. Nombre de la columna que contiene los links
COLUMN_NAME = 'Enlace a Decretos Modificadores'          
# 3. Nombre de la carpeta donde se guardarán los PDFs
DOWNLOAD_FOLDER = 'descargas_pdf'

In [3]:
data = pd.read_excel("ordenanzas.xlsx")

In [4]:
data.head()

,N°,Ordenanza /Decreto,Descripción,Fecha de Promulgación,Enlace,Tipo,Vigente (Si/No),N° Modificación,Fecha Modificación,Enlace a Decretos Modificadores,Documento Completo
0,N° 1749,Decreto,Aprueba Ordenanza sobre Patentes de Expendio y...,2018-11-19,https://drive.google.com/file/d/1bsxzf9BQsp5kT...,Expendio de Bebidas Alcohólicas.,Si,N°2894,2024-12-23,https://drive.google.com/file/d/1HqJMjLRJnGj51...,NaN
1,N° 002,Ordenanza,Aprueba Ordenanza Ambiental para la Comuna de ...,2012-07-04,https://drive.google.com/file/d/1Fm8dhup5kaU3M...,Ordenanza Ambiental,Si,N° 001836,2013-12-24,https://drive.google.com/file/d/1Fdi5SNOOy5Qgc...,NaN
2,NaN,NaN,NaN,NaT,NaN,NaN,Si,N°1413,2016-12-30,NaN,NaN
3,N° 001,Ordenanza,Aprueba Ordenanza sobre Cierro de Vías BNUP,2012-02-03,https://drive.google.com/file/d/16mMX0tyOOsGbq...,Cierres de Pasajes BNUP,Si,N°0125,2023-01-20,https://drive.google.com/file/d/18PNuxLRodoAwU...,NaN
4,N° 001,Ordenanza,Dicta Ordenanza sobre Funcionamiento del Comer...,2002-07-10,https://drive.google.com/file/d/1TnPStsWgGlN2S...,Comercio Estacionado y Ambulante en BNUP,Si,N° 4304,2004-12-17,https://drive.google.com/file/d/1FQmk7oz8RZplU...,NaN


In [5]:
def get_google_drive_file_id(url):
    """Extrae el FILE_ID de varios formatos de link de Google Drive."""
    # Regex para encontrar el ID en formatos /d/ID/view o /file/ID
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)
    match = re.search(r'/file/([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)
    return None

In [6]:
def download_file_from_google_drive(file_id, destination):
    """Descarga un archivo de Google Drive, manejando la confirmación de archivos grandes."""
    URL = "https://drive.google.com/uc?export=download"
    
    session = requests.Session()
    
    # Primera solicitud para obtener la cookie de confirmación
    response = session.get(URL, params={'id': file_id}, stream=True)
    
    token = None
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            token = value
            break
            
    if token:
        # Si hay token, hacer una segunda solicitud con la confirmación
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)
        
    # Guardar el contenido del archivo
    with open(destination, "wb") as f:
        for chunk in response.iter_content(chunk_size=32768):
            if chunk:  # filtrar chunks de 'keep-alive'
                f.write(chunk)

In [20]:
# Celda 4: Definición de Funciones (Actualizada)

def extract_all_file_ids(cell_content):
    """
    Extrae TODOS los File_IDs de un string (celda) que puede contener
    múltiples links de Google Drive.
    """
    # Regex para encontrar /d/ID/ y capturar el ID
    # re.findall() devuelve una lista con todas las coincidencias
    ids = re.findall(r'/d/([a-zA-Z0-9_-]+)', str(cell_content))
    return ids

def download_file_from_google_drive(file_id, destination):
    """Descarga un archivo de Google Drive, manejando la confirmación de archivos grandes."""
    URL = "https://drive.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params={'id': file_id}, stream=True)
    
    token = None
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            token = value
            break
            
    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)
        
    with open(destination, "wb") as f:
        total_length = response.headers.get('content-length')
        if total_length is None: 
            f.writelines(response.iter_lines())
        else:
            total_length = int(total_length)
            # 'leave=False' hace que la barra de progreso se limpie después de terminar
            with tqdm(total=total_length, unit='B', unit_scale=True, desc=os.path.basename(destination), leave=False) as pbar:
                for chunk in response.iter_content(chunk_size=4096):
                    if chunk:
                        f.write(chunk)
                        pbar.update(len(chunk))

In [13]:
# Celda 5: Ejecución Principal

# Crear la carpeta de descargas si no existe
if not os.path.exists(DOWNLOAD_FOLDER):
    os.makedirs(DOWNLOAD_FOLDER)
    print(f"Carpeta '{DOWNLOAD_FOLDER}' creada.")

try:
    # Leer el archivo Excel
    df = pd.read_excel(EXCEL_FILE)
    
    if COLUMN_NAME not in df.columns:
        print(f"Error: No se encontró la columna '{COLUMN_NAME}' en el archivo Excel.")
        print(f"Columnas disponibles: {list(df.columns)}")
    else:
        # Limpiar links (quitar NAs y asegurar que sean strings)
        links = df[COLUMN_NAME].dropna().astype(str).tolist()
        print(f"Se encontraron {len(links)} links para procesar.")
        
        # Iterar sobre los links
        for i, link in enumerate(links):
            print(f"\nProcesando link {i+1}/{len(links)}: {link}")
            file_id = get_google_drive_file_id(link)
            
            if file_id:
                # Usamos un nombre genérico. 
                # Idealmente, tendrías otra columna en el Excel con el nombre deseado.
                file_name = f"archivo_{i+1}.pdf" 
                destination = os.path.join(DOWNLOAD_FOLDER, file_name)
                
                try:
                    download_file_from_google_drive(file_id, destination)
                    print(f"-> ¡Éxito! Guardado como '{destination}'")
                except Exception as e:
                    print(f"-> Error al descargar {link}: {e}")
            else:
                print(f"-> No se pudo extraer un FILE_ID válido de {link}")
                
        print("\n--- Proceso completado ---")
        
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{EXCEL_FILE}'.")
    print("Asegúrate de que esté en la misma carpeta que este notebook.")
except Exception as e:
    print(f"Ha ocurrido un error inesperado: {e}")

Se encontraron 23 links para procesar.

Procesando link 1/23: https://drive.google.com/file/d/1bsxzf9BQsp5kTOdBKSwknjihLGeIP2mL/view


archivo_1.pdf: 100%|██████████| 8.81M/8.81M [00:00<00:00, 13.5MB/s]


-> ¡Éxito! Guardado como 'descargas_pdf\archivo_1.pdf'

Procesando link 2/23: https://drive.google.com/file/d/1Fm8dhup5kaU3M03MRzjQLNSuaK66wiGc/view


archivo_2.pdf: 100%|██████████| 259k/259k [00:00<00:00, 9.38MB/s]


-> ¡Éxito! Guardado como 'descargas_pdf\archivo_2.pdf'

Procesando link 3/23: https://drive.google.com/file/d/16mMX0tyOOsGbq3UtOvlIX8JG_VThu35b/view


...

-> ¡Éxito! Guardado como 'descargas_pdf\archivo_20.pdf'

Procesando link 21/23: https://drive.google.com/file/d/1Ulf03L4H7N6t2iYursWRlL_7QqRmeXpr/view


archivo_21.pdf: 100%|██████████| 26.1k/26.1k [00:00<00:00, 4.88MB/s]


-> ¡Éxito! Guardado como 'descargas_pdf\archivo_21.pdf'

Procesando link 22/23: https://drive.google.com/file/d/1IyWIBU2nri385MjnBzestTAchIe5vGGH/view?usp=drive_link
-> ¡Éxito! Guardado como 'descargas_pdf\archivo_22.pdf'

Procesando link 23/23: https://drive.google.com/file/d/0B-IJxyf1tMZbbXNmWF9HN0MtRUU/view?resourcekey=0-M_RqKflPFUqwHpMofz6MNQ


archivo_23.pdf: 100%|██████████| 203k/203k [00:00<00:00, 5.47MB/s]

-> ¡Éxito! Guardado como 'descargas_pdf\archivo_23.pdf'

--- Proceso completado ---


In [21]:
# Celda 5: Ejecución Principal (Nueva Lógica para Múltiples Links por Celda)

# Crear la carpeta de descargas si no existe
if not os.path.exists(DOWNLOAD_FOLDER):
    os.makedirs(DOWNLOAD_FOLDER)
    print(f"Carpeta '{DOWNLOAD_FOLDER}' creada.")

print("Iniciando proceso de descarga...")

try:
    # 1. Leer TODAS las hojas del archivo Excel
    all_sheets_data = pd.read_excel(EXCEL_FILE, sheet_name=None)
    sheet_names = list(all_sheets_data.keys())
    
    if not sheet_names:
        print("Error: El archivo Excel está vacío, no se encontraron hojas.")
    else:
        print(f"Se encontraron {len(sheet_names)} hojas: {sheet_names}")
        
        # 2. Obtener las hojas a procesar (TODAS EXCEPTO la primera)
        sheets_to_process = sheet_names[1:]
        
        if not sheets_to_process:
            print("Advertencia: Solo se encontró 1 hoja. No hay nada que procesar.")
        else:
            print(f"Procesando {len(sheets_to_process)} hojas (saltando la primera: '{sheet_names[0]}')")
            total_links_descargados = 0

            # 3. Iterar sobre el nombre de cada hoja
            for sheet_name in sheets_to_process:
                print(f"\n--- Leyendo Hoja: '{sheet_name}' ---")
                
                df_sheet = all_sheets_data[sheet_name]
                
                # Limpiamos espacios en blanco de los nombres de columna
                df_sheet.columns = df_sheet.columns.str.strip()
                
                # 4. Verificar si la columna 'COLUMN_NAME' existe en ESTA hoja
                if COLUMN_NAME not in df_sheet.columns:
                    print(f"Advertencia: No se encontró la columna '{COLUMN_NAME}' en la hoja '{sheet_name}'. Saltando esta hoja.")
                    continue
                    
                print(f"  Encontrada columna: '{COLUMN_NAME}'. Buscando links...")
                    
                # 5. Obtener los contenidos de las celdas (y sus índices de fila)
                # .dropna() elimina celdas vacías
                celdas_con_links = df_sheet[COLUMN_NAME].dropna()
                
                if celdas_con_links.empty:
                    print("  -> No se encontraron links en esta columna.")
                    continue
                    
                print(f"  -> Se encontraron {len(celdas_con_links)} celdas con contenido.")
                
                # 6. Iterar sobre CADA CELDA que tiene contenido
                # usamos .items() para obtener el (índice, contenido)
                for fila_indice_pandas, cell_content in celdas_con_links.items():
                    
                    # El índice de pandas es 0-based. Sumamos 2 para el número de fila real de Excel (1-based + cabecera)
                    fila_excel = fila_indice_pandas + 2
                    
                    # 7. !!! NUEVA LÓGICA !!!
                    # Extraer TODOS los IDs de esta celda
                    file_ids = extract_all_file_ids(cell_content)
                    
                    if not file_ids:
                        print(f"    Fila {fila_excel}: No se encontraron IDs de Drive válidos en el texto.")
                        continue
                    
                    print(f"    Fila {fila_excel}: Se encontraron {len(file_ids)} links. Descargando...")
                    
                    # 8. Iterar sobre la LISTA de IDs encontrados en ESTA celda
                    for j, file_id in enumerate(file_ids):
                        # Creamos un nombre de archivo único: Hoja_Fila_LinkNum.pdf
                        sanitized_sheet_name = re.sub(r'[^\w\.-]', '_', sheet_name)
                        file_name = f"{sanitized_sheet_name}_Fila_{fila_excel}_Link_{j+1}.pdf"
                        destination = os.path.join(DOWNLOAD_FOLDER, file_name)
                        
                        try:
                            download_file_from_google_drive(file_id, destination)
                            print(f"      -> ¡Éxito! Link {j+1} guardado como '{destination}'")
                            total_links_descargados += 1
                        except Exception as e:
                            print(f"      -> Error al descargar Link {j+1} (ID: {file_id}): {e}")
            
            print(f"\n--- Proceso completado ---")
            print(f"Se descargaron un total de {total_links_descargados} archivos.")

except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{EXCEL_FILE}'.")
    print("Asegúrate de que esté en la misma carpeta que este notebook.")
except Exception as e:
    print(f"Ha ocurrido un error inesperado: {e}")

Iniciando proceso de descarga...


c:\Users\Cristobal\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Se encontraron 13 hojas: ['Ordenanzas con Modificaciones ', 'Ordenanzas Municipales Sin Modi', 'Ordenanza Secmun', 'Ordenanzas Patentes', 'Ordenanzas Inspección', 'Ordenanza de Genero', 'Ordenanzas Dom', 'Ordenanzas Transito', 'Ordenanzas Dimao', 'Ordenanza Dideco', 'Ordenanza Social', 'Ordenanza JPL', 'Ordenanzas DAF']
Procesando 12 hojas (saltando la primera: 'Ordenanzas con Modificaciones ')

--- Leyendo Hoja: 'Ordenanzas Municipales Sin Modi' ---
Advertencia: No se encontró la columna 'Enlace a Decretos Modificadores' en la hoja 'Ordenanzas Municipales Sin Modi'. Saltando esta hoja.

--- Leyendo Hoja: 'Ordenanza Secmun' ---
Advertencia: No se encontró la columna 'Enlace a Decretos Modificadores' en la hoja 'Ordenanza Secmun'. Saltando esta hoja.

--- Leyendo Hoja: 'Ordenanzas Patentes' ---
Advertencia: No se encontró la columna 'Enlace a Decretos Modificadores' en la hoja 'Ordenanzas Patentes'. Saltando esta hoja.

--- Leyendo Hoja: 'Ordenanzas Inspección' ---
Advertencia: No se en

In [17]:
data.columns

Index(['N°', 'Ordenanza /Decreto ', 'Descripción', 'Fecha de Promulgación ',
       'Enlace', 'Tipo ', 'Vigente (Si/No)', 'N° Modificación ',
       'Fecha Modificación ', 'Enlace a Decretos Modificadores',
       'Documento Completo'],
      dtype='object')

In [25]:
# quiero un df de data que tenga solo la columna "Enlace a Decretos Modificadores"
Data_decretos = data[["Enlace a Decretos Modificadores"]]
Data_decretos.head()

,Enlace a Decretos Modificadores
0,https://drive.google.com/file/d/1HqJMjLRJnGj51...
1,https://drive.google.com/file/d/1Fdi5SNOOy5Qgc...
2,NaN
3,https://drive.google.com/file/d/18PNuxLRodoAwU...
4,https://drive.google.com/file/d/1FQmk7oz8RZplU...


In [27]:
# Quiero varios df del excel (que tiene 12 paginas) y que cada df solo tenga la columna "Enlace a Decretos Modificadores", y si no tierne esa columna, que no haga nada con esa pagina
import pandas as pd

# Leer el archivo Excel
xls = pd.ExcelFile("ordenanzas.xlsx")

# Crear un diccionario para almacenar los DataFrames
dataframes = {}

# Iterar sobre cada hoja del Excel
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    # Verificar si la columna existe
    if "Enlace a Decretos Modificadores" in df.columns:
        dataframes[sheet_name] = df[["Enlace a Decretos Modificadores"]]

# Mostrar los DataFrames resultantes
for name, df in dataframes.items():
    print(f"--- {name} ---")
    print(df.head())

c:\Users\Cristobal\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


--- Ordenanzas con Modificaciones  ---
                     Enlace a Decretos Modificadores
0  https://drive.google.com/file/d/1HqJMjLRJnGj51...
1  https://drive.google.com/file/d/1Fdi5SNOOy5Qgc...
2                                                NaN
3  https://drive.google.com/file/d/18PNuxLRodoAwU...
4  https://drive.google.com/file/d/1FQmk7oz8RZplU...


In [30]:
print(dataframes)

{'Ordenanzas con Modificaciones ':                       Enlace a Decretos Modificadores
0   https://drive.google.com/file/d/1HqJMjLRJnGj51...
1   https://drive.google.com/file/d/1Fdi5SNOOy5Qgc...
2                                                 NaN
3   https://drive.google.com/file/d/18PNuxLRodoAwU...
4   https://drive.google.com/file/d/1FQmk7oz8RZplU...
..                                                ...
76  https://drive.google.com/file/d/19pp3JPlT5g4pJ...
77  https://drive.google.com/file/d/1x73spkBQdd89V...
78  https://drive.google.com/file/d/1qeTk2lA-pzOsR...
79  https://drive.google.com/file/d/1_TyiHGHAuCPvL...
80  https://drive.google.com/file/d/1HKQtXgCLnAPu9...

[81 rows x 1 columns]}


In [29]:
len(df)

81

In [37]:
# Celda 1: Importar Bibliotecas y Configurar
import pandas as pd
import re
import requests
import os
from tqdm.auto import tqdm # Para las barras de progreso

# 1. Nombre de tu archivo Excel
EXCEL_FILE = 'ordenanzas.xlsx' 

# 2. LISTA de columnas a buscar (en orden de prioridad)
COLUMNS_TO_SEARCH = [
    'Enlace con ultima modificación',
    'Enlace a Decretos Modificadores'
]

# 3. Nombre de la carpeta donde se guardarán los PDFs
DOWNLOAD_FOLDER = 'descargas_pdf'

In [38]:
# Celda 2: Lógica para crear los DataFrames

# Este diccionario guardará tus nuevos DataFrames
dataframes_extraidos = {}
df_counter = 1 

print("Iniciando Fase 1: Extrayendo DataFrames del Excel...")

try:
    all_sheets_data = pd.read_excel(EXCEL_FILE, sheet_name=None)
    print(f"Se encontraron {len(all_sheets_data)} hojas en total.")

    for sheet_name, df_sheet in all_sheets_data.items():
        df_sheet.columns = df_sheet.columns.str.strip()
        
        columna_encontrada = None
        for col_name in COLUMNS_TO_SEARCH:
            if col_name in df_sheet.columns:
                columna_encontrada = col_name
                break 
        
        if columna_encontrada:
            # Creamos el df solo con la columna, quitando NAs
            df_final = df_sheet[[columna_encontrada]].dropna().copy()
            
            # Renombramos la columna a 'Enlace' para que sea estándar
            df_final.columns = ['Enlace']
            
            if not df_final.empty:
                # Usamos el nombre de la hoja original para el nombre del df
                # Limpiamos el nombre de la hoja para que sea un nombre de archivo seguro
                sanitized_sheet_name = re.sub(r'[^\w\.-]', '_', sheet_name)
                
                # Guardar el DataFrame en nuestro diccionario
                dataframes_extraidos[sanitized_sheet_name] = df_final
                
                print(f"-> ÉXITO: Hoja '{sheet_name}' procesada. Creado DataFrame '{sanitized_sheet_name}' con {len(df_final)} links.")
                
            else:
                print(f"-> Info: Hoja '{sheet_name}' (col '{columna_encontrada}') no tiene links. Omitiendo.")
                
        else:
            print(f"-> Info: Hoja '{sheet_name}' omitida (no se encontró ninguna columna buscada).")

    print(f"\n--- Fase 1 Completada ---")
    print(f"Se crearon {len(dataframes_extraidos)} DataFrames en 'dataframes_extraidos'.")

except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{EXCEL_FILE}'.")
except Exception as e:
    print(f"Ha ocurrido un error inesperado: {e}")

Iniciando Fase 1: Extrayendo DataFrames del Excel...


c:\Users\Cristobal\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Se encontraron 13 hojas en total.
-> ÉXITO: Hoja 'Ordenanzas con Modificaciones ' procesada. Creado DataFrame 'Ordenanzas_con_Modificaciones_' con 80 links.
-> Info: Hoja 'Ordenanzas Municipales Sin Modi' omitida (no se encontró ninguna columna buscada).
-> ÉXITO: Hoja 'Ordenanza Secmun' procesada. Creado DataFrame 'Ordenanza_Secmun' con 1 links.
-> ÉXITO: Hoja 'Ordenanzas Patentes' procesada. Creado DataFrame 'Ordenanzas_Patentes' con 3 links.
-> ÉXITO: Hoja 'Ordenanzas Inspección' procesada. Creado DataFrame 'Ordenanzas_Inspección' con 9 links.
-> ÉXITO: Hoja 'Ordenanza de Genero' procesada. Creado DataFrame 'Ordenanza_de_Genero' con 1 links.
-> ÉXITO: Hoja 'Ordenanzas Dom' procesada. Creado DataFrame 'Ordenanzas_Dom' con 4 links.
-> ÉXITO: Hoja 'Ordenanzas Transito' procesada. Creado DataFrame 'Ordenanzas_Transito' con 3 links.
-> ÉXITO: Hoja 'Ordenanzas Dimao' procesada. Creado DataFrame 'Ordenanzas_Dimao' con 3 links.
-> ÉXITO: Hoja 'Ordenanza Dideco' procesada. Creado DataFrame '

In [41]:
# Celda 3: Funciones de Ayuda para Descarga (CORREGIDA)

def extract_all_file_ids(cell_content):
    """Extrae TODOS los File_IDs de un string (celda)."""
    # Busca IDs tanto en el formato /d/ID/ como en ?id=ID
    ids1 = re.findall(r'/d/([a-zA-Z0-9_-]{28,})', str(cell_content))
    ids2 = re.findall(r'id=([a-zA-Z0-9_-]{28,})', str(cell_content))
    # Combina las listas y elimina duplicados
    all_ids = list(dict.fromkeys(ids1 + ids2))
    return all_ids

def download_file_from_google_drive(file_id, destination):
    """
    Descarga un archivo de Google Drive, manejando la confirmación de 
    archivos grandes y verificando que sea un PDF.
    """
    URL = "https://drive.google.com/uc?export=download"
    session = requests.Session()
    
    # --- 1. Lógica de confirmación de archivo grande ---
    response = session.get(URL, params={'id': file_id}, stream=True)
    
    token = None
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            token = value
            break
            
    if token:
        # Si hubo advertencia, hacer la 2da solicitud con la confirmación
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)
    
    # --- 2. ¡NUEVA VERIFICACIÓN! ---
    # Asegurarnos de que Google nos está enviando un PDF y no una página de error HTML
    content_type = response.headers.get('content-type', '').lower()
    
    if 'application/pdf' not in content_type:
        # Si no es un PDF, es un error (login, cuota, etc.)
        # No guardamos el archivo y lanzamos un error que la Celda 4 capturará.
        if 'text/html' in content_type:
            raise Exception(f"No se recibió un PDF. Se recibió HTML. Causa probable: Archivo privado, requiere login o se superó la cuota de descarga.")
        else:
            raise Exception(f"Tipo de contenido inesperado: '{content_type}'. No es un PDF.")

    # --- 3. Lógica de guardado (CORREGIDA) ---
    # Siempre usamos iter_content() para archivos binarios.
    with open(destination, "wb") as f:
        total_length = response.headers.get('content-length')
        
        if total_length is None:
            # No hay barra de progreso, solo guardamos
            print(f"    (Descargando {os.path.basename(destination)}, tamaño desconocido)...", end='')
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
            print(" ¡Hecho!")
        else:
            # Hay barra de progreso
            total_length = int(total_length)
            with tqdm(total=total_length, unit='B', unit_scale=True, desc=os.path.basename(destination), leave=False) as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        pbar.update(len(chunk))

In [42]:
# Celda 4: Lógica de Descarga

# Crear la carpeta de descargas si no existe
if not os.path.exists(DOWNLOAD_FOLDER):
    os.makedirs(DOWNLOAD_FOLDER)
    print(f"Carpeta '{DOWNLOAD_FOLDER}' creada.")

print("\nIniciando Fase 2: Descargando archivos...")

if not dataframes_extraidos:
    print("Error: No hay DataFrames en 'dataframes_extraidos' para procesar. Ejecuta la Celda 2 primero.")
else:
    total_links_descargados = 0
    
    # 1. Iterar sobre cada DataFrame que creamos (ej. 'Hoja_Limpia', df_con_links)
    for df_name, df_links in dataframes_extraidos.items():
        
        print(f"\n--- Procesando DataFrame: '{df_name}' ---")
        
        # 2. Iterar sobre CADA CELDA en la columna 'Enlace' de este DataFrame
        # (usamos .iterrows() para obtener el índice de la fila y el contenido)
        for fila_indice, fila_data in df_links.iterrows():
            
            cell_content = fila_data['Enlace'] # Obtener el texto de la celda
            
            # 3. Extraer TODOS los IDs de esa celda
            file_ids = extract_all_file_ids(cell_content)
            
            if not file_ids:
                print(f"    Fila {fila_indice + 2}: No se encontraron IDs de Drive válidos.")
                continue
                
            print(f"    Fila {fila_indice + 2}: Se encontraron {len(file_ids)} links. Descargando...")
            
            # 4. Iterar sobre la LISTA de IDs encontrados en ESTA celda
            for j, file_id in enumerate(file_ids):
                
                # 5. Crear el nombre del archivo: Hoja_Modificacion_X.pdf
                # Usamos el nombre del df (que es el nombre de la hoja limpio)
                # 'j+1' será 1, 2, 3...
                file_name = f"{df_name}_Modificacion_{j+1}.pdf"
                
                # --- AJUSTE IMPORTANTE ---
                # Si una celda tiene un link, y otra celda después tiene otro,
                # ambos se llamarían "Modificacion_1".
                # Necesitamos un contador por DataFrame.
                
                # Descomenta la siguiente línea para el nombre original
                # file_name = f"{df_name}_Fila_{fila_indice+2}_Modif_{j+1}.pdf"
                
                # --- Usaremos un contador global por HOJA ---
                # Vamos a renombrar el bucle para hacerlo más simple
                
                pass # Eliminamos la lógica anterior para reemplazarla

        # --- LÓGICA CORREGIDA (MEJOR) ---
        
        # Reiniciamos el contador de modificaciones para CADA hoja
        contador_modificacion = 1 
        
        # Iteramos sobre el contenido de las celdas
        for cell_content in df_links['Enlace']:
            
            # Extraemos todos los IDs de la celda
            file_ids = extract_all_file_ids(cell_content)
            
            if not file_ids:
                continue # Esta celda no tenía IDs válidos
            
            # Iteramos sobre los IDs encontrados en esa celda
            for file_id in file_ids:
                
                # Creamos el nombre secuencial
                file_name = f"{df_name}_Modificacion_{contador_modificacion}.pdf"
                destination = os.path.join(DOWNLOAD_FOLDER, file_name)
                
                print(f"  -> Descargando '{file_name}'...")
                
                try:
                    download_file_from_google_drive(file_id, destination)
                    print(f"    ¡Éxito! Guardado como '{destination}'")
                    total_links_descargados += 1
                    
                    # MUY IMPORTANTE: Solo incrementamos el contador si la descarga fue exitosa
                    contador_modificacion += 1 
                    
                except Exception as e:
                    print(f"    -> Error al descargar (ID: {file_id}): {e}")
            
    print(f"\n--- Fase 2 Completada ---")
    print(f"Se descargaron un total de {total_links_descargados} archivos.")

Carpeta 'descargas_pdf' creada.

Iniciando Fase 2: Descargando archivos...

--- Procesando DataFrame: 'Ordenanzas_con_Modificaciones_' ---
    Fila 2: Se encontraron 1 links. Descargando...
    Fila 3: Se encontraron 1 links. Descargando...
    Fila 5: Se encontraron 1 links. Descargando...
    Fila 6: Se encontraron 1 links. Descargando...
    Fila 7: Se encontraron 1 links. Descargando...
    Fila 8: Se encontraron 1 links. Descargando...
    Fila 9: Se encontraron 1 links. Descargando...
    Fila 10: Se encontraron 1 links. Descargando...
    Fila 11: Se encontraron 1 links. Descargando...
    Fila 12: Se encontraron 1 links. Descargando...
    Fila 13: Se encontraron 1 links. Descargando...
    Fila 14: Se encontraron 1 links. Descargando...
    Fila 15: Se encontraron 1 links. Descargando...
    Fila 16: No se encontraron IDs de Drive válidos.
    Fila 17: Se encontraron 1 links. Descargando...
    Fila 18: Se encontraron 1 links. Descargando...
    Fila 19: Se encontraron 1 links

KeyboardInterrupt: 

In [11]:
import pytesseract
from pdf2image import convert_from_path
import os

In [12]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [13]:
try:
    # Verificar que Tesseract está instalado y accesible
    version = pytesseract.get_tesseract_version()
    print(f"✅ Tesseract está instalado correctamente. Versión: {version}")
    
    # Verificar que el archivo existe
    if os.path.exists(pytesseract.pytesseract.tesseract_cmd):
        print(f"✅ El ejecutable de Tesseract existe en la ruta especificada")
    else:
        print(f"❌ No se encuentra el ejecutable de Tesseract en la ruta: {pytesseract.pytesseract.tesseract_cmd}")
except Exception as e:
    print(f"❌ Error al verificar Tesseract: {e}")

✅ Tesseract está instalado correctamente. Versión: 5.5.0.20241111
✅ El ejecutable de Tesseract existe en la ruta especificada


In [7]:
## DECRETO 2894- APRUEBA MODIFICACIÓN DE ORDENANZA SOBRE PATENTES DE EXPENDIOY CONSUMO DE BEBIDAS ALCOHOLIDAS DE LA COMUNA DE RENCA, APROBADA POR DECRETO ALCALDICIO N 1749 DE FECHA 18 DE

carpeta_pdf = "prueba"
# Nombre del PDF de prueba (ajusta este nombre)
nombre_pdf = "DECRETO 2894- APRUEBA MODIFICACIÓN DE ORDENANZA SOBRE PATENTES DE EXPENDIOY CONSUMO DE BEBIDAS ALCOHOLIDAS DE LA COMUNA DE RENCA, APROBADA POR DECRETO ALCALDICIO N 1749 DE FECHA 18 DE.pdf" 
ruta_pdf = os.path.join(carpeta_pdf, nombre_pdf)

In [8]:
if not os.path.exists(ruta_pdf):
    print(f"🚨 ERROR: No se encontró el archivo en la ruta: {ruta_pdf}")
else:
    print(f"✅ Iniciando procesamiento OCR para: {ruta_pdf}")
    
    # --- 1. Conversión de PDF a Imágenes ---
    try:
        # Convierte el PDF en una lista de objetos Image (uno por página)
        # 'dpi=300' es una buena práctica para mejorar la precisión del OCR
        paginas = convert_from_path(ruta_pdf, dpi=300)
        
        texto_completo = ""
        
        # --- 2. Iteración y Aplicación de OCR ---
        for i, pagina in enumerate(paginas):
            print(f"   -> Procesando Página {i + 1}...")
            
            # Aplica Tesseract OCR a la imagen de la página
            # 'lang='spa'' especifica el idioma español para mejorar la precisión
            texto_pagina = pytesseract.image_to_string(pagina, lang='spa')
            
            # Concatena el texto, añadiendo un separador de página
            texto_completo += f"\n\n--- INICIO PÁGINA {i + 1} ---\n"
            texto_completo += texto_pagina
            texto_completo += f"\n--- FIN PÁGINA {i + 1} ---\n"
            
        print("\n🎉 OCR Completado. Mostrando extracto del texto:")
        
        # --- 3. Mostrar y Guardar Resultado ---
        # Muestra las primeras 500 caracteres del texto extraído
        print("-" * 30)
        print(texto_completo[:500] + "...") 
        print("-" * 30)
        
        # Guarda el texto extraído en un archivo .txt
        nombre_txt = nombre_pdf.replace(".pdf", "_OCR.txt")
        ruta_txt = os.path.join(carpeta_pdf, nombre_txt)
        
        with open(ruta_txt, "w", encoding="utf-8") as f:
            f.write(texto_completo)
            
        print(f"\n💾 Texto guardado en: {ruta_txt}")

    except pytesseract.TesseractNotFoundError:
        print("\n❌ ERROR: El motor Tesseract no está instalado o su ruta no está configurada.")
        print("Asegúrate de instalar Tesseract en tu sistema operativo.")
    except Exception as e:
        print(f"\n❌ Ocurrió un error durante el procesamiento: {e}")

✅ Iniciando procesamiento OCR para: prueba\DECRETO 2894- APRUEBA MODIFICACIÓN DE ORDENANZA SOBRE PATENTES DE EXPENDIOY CONSUMO DE BEBIDAS ALCOHOLIDAS DE LA COMUNA DE RENCA, APROBADA POR DECRETO ALCALDICIO N 1749 DE FECHA 18 DE.pdf
   -> Procesando Página 1...
   -> Procesando Página 1...
   -> Procesando Página 2...
   -> Procesando Página 2...
   -> Procesando Página 3...
   -> Procesando Página 3...
   -> Procesando Página 4...
   -> Procesando Página 4...
   -> Procesando Página 5...
   -> Procesando Página 5...
   -> Procesando Página 6...
   -> Procesando Página 6...
   -> Procesando Página 7...
   -> Procesando Página 7...
   -> Procesando Página 8...
   -> Procesando Página 8...

🎉 OCR Completado. Mostrando extracto del texto:
------------------------------


--- INICIO PÁGINA 1 ---
SECRETARÍA MUNICIPAL

DECRETO ALCALDICIO N22894 — 23 DIC 2024

MATERIA: Aprueba Modificación de Ordenanza
sobre Patentes de Expendio y Consumo de Bebidas
Alcohólicas de la Comuna de Renca, aprobada p

In [9]:
texto_completo[:1000]

'\n\n--- INICIO PÁGINA 1 ---\nSECRETARÍA MUNICIPAL\n\nDECRETO ALCALDICIO N22894 — 23 DIC 2024\n\nMATERIA: Aprueba Modificación de Ordenanza\nsobre Patentes de Expendio y Consumo de Bebidas\nAlcohólicas de la Comuna de Renca, aprobada por\nDecreto Alcaldicio N* 1749 de fecha 19 de noviembre\nde 2018\n\nVISTOS:\n\nLas facultades establecidas en la Ley N2 18.695, Orgánica Constitucional de Municipalidades, la Ley 19.886\nde Bases sobre contratos administrativos de suministro y prestación de servicios y su Reglamento y el\nDecreto Alcaldicio N” 833 de fecha 28 de agosto de 2020 y sus modificaciones, que delega facultades a\ndiversos funcionarios municipales; y\n\nCONSIDERANDO:\n\n1.- El Decreto Alcaldicio N* 1749 de fecha 19 de noviembre de 2018 que aprueba Ordenanza sobre Patentes\nde Expendio y Consumo de Bebidas Alcohólicas de la Comuna de Renca.\n\n2.- El Certificado N* 267 de fecha 27 de noviembre de 2024 en que la Secretaria Municipal de Renca, en su\ncalidad de Ministro de Fe de los

In [10]:
!pip install google-api-python-client google-auth-oauthlib google-auth-httplib2 pandas openpyxl

  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ------- -------------------------------- 2.6/14.6 MB 14.1 MB/s eta 0:00:01
   ------------------------- -------------- 9.2/14.6 MB 23.2 MB/s eta 0:00:01
   ---------------------------------------- 14.6/14.6 MB 27.0 MB/s eta 0:00:00
Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
Using cached pyasn1-0.6.1-py3-none-any.whl (83 kB)
Using cached pyasn1_modules-0.4.2-py3-none-any.whl (181 kB)
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)

   -- -------------------------------------  1/16 [pyasn1]
   -- -------------------------------------  1/16 [pyasn1]
   -- -------------------------------------  1/16 [pyasn1]
   ----- ------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import os
import io
import re  

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload


ARCHIVO_SECRETO_CLIENTE = 'client_secret.json'
ARCHIVO_EXCEL = 'ordenanzas.xlsx'  
COLUMNA_LINKS = 'Enlace a Decretos Modificadores' 
CARPETA_DESCARGA = 'modificaciones_descargadas'  

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']


if not os.path.exists(CARPETA_DESCARGA):
    os.makedirs(CARPETA_DESCARGA)

In [ ]:
def autenticar_google():
    """
    Realiza la autenticación con Google Drive.
    Crea token.json la primera vez, y lo reutiliza después.
    """
    creds = None
    # El archivo token.json almacena los tokens de acceso y actualización del usuario.
    # Se crea automáticamente la primera vez que se completa el flujo de autorización.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # Si no hay credenciales (válidas) disponibles, deja que el usuario inicie sesión.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # ¡AQUÍ ES DONDE SE USA TU JSON!
            flow = InstalledAppFlow.from_client_secrets_file(
                ARCHIVO_SECRETO_CLIENTE, SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Guarda las credenciales para la próxima ejecución
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    print("Autenticación exitosa.")
    return creds

# Ejecutamos la autenticación
try:
    creds = autenticar_google()
    # Construimos el "servicio" que nos permite hablar con la API de Drive
    drive_service = build('drive', 'v3', credentials=creds)
    print("Servicio de Google Drive listo.")

except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{ARCHIVO_SECRETO_CLIENTE}'.")
    print("Asegúrate de que esté en la misma carpeta que tu notebook.")
except Exception as e:
    print(f"Ocurrió un error durante la autenticación: {e}")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=669567508181-n4438p56n65dqih4n090rlaa6en7svaa.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59954%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=9qFlycoQ13vmbNPCE3U5BsEJX7tbZ0&access_type=offline
Autenticación exitosa.
Servicio de Google Drive listo.
Autenticación exitosa.
Servicio de Google Drive listo.


In [ ]:
def extraer_id_de_link(url):
    """
    Extrae el ID de archivo de un link de Google Drive usando expresiones regulares.
    Maneja varios formatos de link comunes.
    """
    if not isinstance(url, str):
        return None
        
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)

    match = re.search(r'[?&]id=([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)
        
    return None

print(f"ID de '.../d/123_abc/edit': {extraer_id_de_link('.../d/123_abc/edit')}")
print(f"ID de '...?id=456-xyz': {extraer_id_de_link('...?id=456-xyz')}")

ID de '.../d/123_abc/edit': 123_abc
ID de '...?id=456-xyz': 456-xyz


In [ ]:
print(f"Iniciando el proceso de descarga...")

try:
    # 1. Leer TODAS las hojas del archivo Excel
    all_sheets = pd.read_excel(ARCHIVO_EXCEL, sheet_name=None)
    total_archivos_descargados = 0
    
    print(f"Se encontraron {len(all_sheets)} hojas en el Excel")
    
    # Iterar sobre cada hoja
    for sheet_name, df in all_sheets.items():
        print(f"\n=== Procesando hoja: {sheet_name} ===")
        
        if COLUMNA_LINKS not in df.columns:
            print(f"Info: No se encontró la columna '{COLUMNA_LINKS}' en la hoja '{sheet_name}'. Saltando...")
            continue
            
        # Obtener la lista de links (eliminando valores vacíos)
        links = df[COLUMNA_LINKS].dropna().tolist()
        total_links = len(links)
        print(f"Se encontraron {total_links} links en la hoja '{sheet_name}'")
        
        # Crear subcarpeta para esta hoja
        carpeta_hoja = os.path.join(CARPETA_DESCARGA, re.sub(r'[^\w\.-]', '_', sheet_name))
        if not os.path.exists(carpeta_hoja):
            os.makedirs(carpeta_hoja)
            print(f"Creada subcarpeta: {carpeta_hoja}")
        
        # Iterar y descargar
        for i, link in enumerate(links, 1):
            print(f"\n--- Procesando link {i} de {total_links} de la hoja {sheet_name} ---")
            print(f"URL: {link}")
            
            # Extraer el ID
            file_id = extraer_id_de_link(link)
            
            if not file_id:
                print(f"No se pudo extraer un ID de Google Drive de este link. Saltando.")
                continue
                
            print(f"ID de Drive extraído: {file_id}")
            
            try:
                # Preparar la solicitud de descarga a la API
                request = drive_service.files().get_media(fileId=file_id)
                
                # Obtener metadatos del archivo para conseguir el nombre original
                file_metadata = drive_service.files().get(fileId=file_id, fields='name, mimeType').execute()
                original_filename = file_metadata.get('name', f'archivo_descargado_{i}.pdf')
                
                # Verificar que sea un PDF
                mime_type = file_metadata.get('mimeType', '')
                if 'pdf' not in mime_type.lower():
                    print(f"⚠️ Advertencia: El archivo no parece ser un PDF (tipo: {mime_type})")
                
                # Asegurar que el nombre del archivo sea válido
                safe_filename = re.sub(r'[^\w\.-]', '_', original_filename)
                if not safe_filename.lower().endswith('.pdf'):
                    safe_filename += '.pdf'
                
                output_path = os.path.join(carpeta_hoja, f"{i:03d}_{safe_filename}")
                
                # Usar un buffer de memoria para descargar
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                
                done = False
                print(f"📥 Descargando {safe_filename}...")
                while done is False:
                    status, done = downloader.next_chunk()
                    if status:
                        print(f"    Progreso: {int(status.progress() * 100)}%", end='\r')
                
                # Guardar el archivo descargado en el disco
                fh.seek(0)
                with open(output_path, 'wb') as f:
                    f.write(fh.read())
                    
                print(f"\n✅ ¡Éxito! Guardado como: {output_path}")
                total_archivos_descargados += 1

            except Exception as e:
                print(f"❌ ERROR al descargar el archivo con ID {file_id}")
                print(f"   Detalle del error: {str(e)}")
                print("   Posibles causas: ¿Tienes permiso? ¿Es un link válido?")
        
        print(f"\n=== Completada la hoja {sheet_name} ===")

    print("\n=== ¡Proceso de descarga completado! ===")
    print(f"Total de archivos descargados: {total_archivos_descargados}")
    print(f"Los archivos están organizados por hoja en subcarpetas dentro de '{CARPETA_DESCARGA}'")

except FileNotFoundError:
    print(f"Error: No se encontró el archivo Excel '{ARCHIVO_EXCEL}'.")
    print("Asegúrate de que el nombre es correcto y está en la misma carpeta.")
except Exception as e:
    print(f"Ocurrió un error general: {e}")

Iniciando el proceso de descarga...


c:\Users\Cristobal\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Se encontraron 13 hojas en el Excel

=== Procesando hoja: Ordenanzas con Modificaciones  ===
Se encontraron 80 links en la hoja 'Ordenanzas con Modificaciones '

--- Procesando link 1 de 80 de la hoja Ordenanzas con Modificaciones  ---
URL: https://drive.google.com/file/d/1HqJMjLRJnGj51iwmRYfenAZ1WPj9-arL/view?usp=drive_link
ID de Drive extraído: 1HqJMjLRJnGj51iwmRYfenAZ1WPj9-arL
📥 Descargando DECRETO_2894-_APRUEBA_MODIFICACIÓN_DE_ORDENANZA_SOBRE_PATENTES_DE_EXPENDIOY_CONSUMO_DE_BEBIDAS_ALCOHOLIDAS_DE_LA_COMUNA_DE_RENCA__APROBADA_POR_DECRETO_ALCALDICIO_N_1749_DE_FECHA_18_DE.pdf...
📥 Descargando DECRETO_2894-_APRUEBA_MODIFICACIÓN_DE_ORDENANZA_SOBRE_PATENTES_DE_EXPENDIOY_CONSUMO_DE_BEBIDAS_ALCOHOLIDAS_DE_LA_COMUNA_DE_RENCA__APROBADA_POR_DECRETO_ALCALDICIO_N_1749_DE_FECHA_18_DE.pdf...
    Progreso: 100%
✅ ¡Éxito! Guardado como: modificaciones_descargadas\Ordenanzas_con_Modificaciones_\001_DECRETO_2894-_APRUEBA_MODIFICACIÓN_DE_ORDENANZA_SOBRE_PATENTES_DE_EXPENDIOY_CONSUMO_DE_BEBIDAS_ALCOHO

In [15]:
# Importar bibliotecas necesarias
import os
import pandas as pd
from pdf2image import convert_from_path
import pytesseract
from tqdm.auto import tqdm

def procesar_pdf(ruta_pdf):
    """
    Procesa un archivo PDF y extrae su texto usando OCR.
    Retorna el texto completo extraído.
    """
    try:
        # Convertir PDF a imágenes
        paginas = convert_from_path(ruta_pdf, dpi=300)
        texto_completo = ""
        
        # Procesar cada página
        for i, pagina in enumerate(paginas):
            # Aplicar OCR
            texto_pagina = pytesseract.image_to_string(pagina, lang='spa')
            texto_completo += texto_pagina + "\n"
            
        return texto_completo.strip()
    
    except Exception as e:
        return f"Error al procesar {os.path.basename(ruta_pdf)}: {str(e)}"

# Configuración
carpeta_pdfs = "modificaciones_descargadas"  # Carpeta donde están los PDFs
archivo_excel_salida = "textos_extraidos.xlsx"  # Nombre del archivo Excel de salida

# Obtener lista de PDFs en la carpeta
pdfs = [f for f in os.listdir(carpeta_pdfs) if f.lower().endswith('.pdf')]

if not pdfs:
    print("❌ No se encontraron archivos PDF en la carpeta especificada.")
else:
    print(f"✅ Se encontraron {len(pdfs)} archivos PDF para procesar.")
    
    # Crear un diccionario para almacenar los resultados
    resultados = {
        'Nombre_Archivo': [],
        'Texto_Extraido': []
    }
    
    # Procesar cada PDF
    for pdf in tqdm(pdfs, desc="Procesando PDFs"):
        ruta_completa = os.path.join(carpeta_pdfs, pdf)
        
        # Extraer texto
        texto = procesar_pdf(ruta_completa)
        
        # Guardar resultados
        resultados['Nombre_Archivo'].append(pdf)
        resultados['Texto_Extraido'].append(texto)
    
    # Crear DataFrame y guardar en Excel
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel(archivo_excel_salida, index=False)
    
    print(f"\n✅ Proceso completado. Los textos han sido guardados en '{archivo_excel_salida}'")
    print(f"   Se procesaron {len(pdfs)} archivos PDF.")
    
    # Mostrar las primeras filas del DataFrame
    print("\nPrimeras entradas del Excel generado:")
    print(df_resultados.head())

✅ Se encontraron 99 archivos PDF para procesar.


Procesando PDFs: 100%|██████████| 99/99 [07:10<00:00,  4.35s/it]


✅ Proceso completado. Los textos han sido guardados en 'textos_extraidos.xlsx'
   Se procesaron 99 archivos PDF.

Primeras entradas del Excel generado:
                                      Nombre_Archivo  \
0  001_DECRETO_2894-_APRUEBA_MODIFICACIÓN_DE_ORDE...   
1  002_DECRETO_1836-_MODIFICA_ORDENANZA_AMBIENTAL...   
2  003_DECRETO_125-_APRUEBA_ORDENANZA_MUNICIPAL_P...   
3  004_DECRETO_4304-_MODIFICA_ORDENANZA_N_001_SOB...   
4  005_DECRETO_987-_MODIFICA_ORDENANZA_SOBRE_FUNC...   

                                      Texto_Extraido  
0  Error al procesar 001_DECRETO_2894-_APRUEBA_MO...  
1  . DOig836 y Y4DIC 20\nREPUBLICA DE CHILE DECRE...  
2  Error al procesar 003_DECRETO_125-_APRUEBA_ORD...  
3  DECRETO N* 434 47. 2004\n\nMAT.: Modifica Orde...  
4  REPUBLICA DE CHILE DECRETO N* y /\n| MUNICIPAL...  


In [16]:
df2 = pd.read_excel("textos_extraidos.xlsx")

In [17]:
df2.head()

,Nombre_Archivo,Texto_Extraido
0,001_DECRETO_2894-_APRUEBA_MODIFICACIÓN_DE_ORDE...,Error al procesar 001_DECRETO_2894-_APRUEBA_MO...
1,002_DECRETO_1836-_MODIFICA_ORDENANZA_AMBIENTAL...,. DOig836 y Y4DIC 20\nREPUBLICA DE CHILE DECRE...
2,003_DECRETO_125-_APRUEBA_ORDENANZA_MUNICIPAL_P...,Error al procesar 003_DECRETO_125-_APRUEBA_ORD...
3,004_DECRETO_4304-_MODIFICA_ORDENANZA_N_001_SOB...,DECRETO N* 434 47. 2004\n\nMAT.: Modifica Orde...
4,005_DECRETO_987-_MODIFICA_ORDENANZA_SOBRE_FUNC...,REPUBLICA DE CHILE DECRETO N* y /\n| MUNICIPAL...


In [18]:
# Importar bibliotecas necesarias


def procesar_pdf(ruta_pdf):
    """
    Procesa un archivo PDF y extrae su texto usando OCR.
    Retorna el texto completo extraído.
    """
    try:
        # Convertir PDF a imágenes
        paginas = convert_from_path(ruta_pdf, dpi=300)
        texto_completo = ""
        
        # Procesar cada página
        for i, pagina in enumerate(paginas):
            # Aplicar OCR
            texto_pagina = pytesseract.image_to_string(pagina, lang='spa')
            texto_completo += texto_pagina + "\n"
            
        return texto_completo.strip()
    
    except Exception as e:
        return f"Error al procesar {os.path.basename(ruta_pdf)}: {str(e)}"

# Configuración
carpeta_pdfs = "ordenanzas_sin_modificaciones"  # Carpeta donde están los PDFs
archivo_excel_salida = "ordenanzas_sin_modificaciones_texto.xlsx"  # Nombre del archivo Excel de salida

# Obtener lista de PDFs en la carpeta
pdfs = [f for f in os.listdir(carpeta_pdfs) if f.lower().endswith('.pdf')]

if not pdfs:
    print("❌ No se encontraron archivos PDF en la carpeta especificada.")
else:
    print(f"✅ Se encontraron {len(pdfs)} archivos PDF para procesar.")
    
    # Crear un diccionario para almacenar los resultados
    resultados = {
        'Nombre_Archivo': [],
        'Texto_Extraido': []
    }
    
    # Procesar cada PDF
    for pdf in tqdm(pdfs, desc="Procesando PDFs"):
        ruta_completa = os.path.join(carpeta_pdfs, pdf)
        
        # Extraer texto
        texto = procesar_pdf(ruta_completa)
        
        # Guardar resultados
        resultados['Nombre_Archivo'].append(pdf)
        resultados['Texto_Extraido'].append(texto)
    
    # Crear DataFrame y guardar en Excel
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel(archivo_excel_salida, index=False)
    
    print(f"\n✅ Proceso completado. Los textos han sido guardados en '{archivo_excel_salida}'")
    print(f"   Se procesaron {len(pdfs)} archivos PDF.")
    
    # Mostrar las primeras filas del DataFrame
    print("\nPrimeras entradas del Excel generado:")
    print(df_resultados.head())

✅ Se encontraron 62 archivos PDF para procesar.


Procesando PDFs: 100%|██████████| 62/62 [14:50<00:00, 14.37s/it]


✅ Proceso completado. Los textos han sido guardados en 'ordenanzas_sin_modificaciones_texto.xlsx'
   Se procesaron 62 archivos PDF.

Primeras entradas del Excel generado:
   Nombre_Archivo                                     Texto_Extraido
0   archivo_1.pdf  REPUBLICA DE CHILE Decreto NA U 1 74 9 19 NOV ...
1  archivo_10.pdf  - 37 -\n\nREPUBLICA DE CHILE\n!. MUNICIPALIDAD...
2  archivo_11.pdf  - 111 -\n\nREPUBLICA DE CHILE\n!. MUNICIPALIDA...
3  archivo_12.pdf  - 98 -\n\nREPUBLICA DE CHILE\n!. MUNICIPALIDAD...
4  archivo_13.pdf  - 40 -\n\nREPUBLICA DE CHILE\nI. MUNICIPALIDAD...


In [23]:
# Importar bibliotecas necesarias


def procesar_pdf(ruta_pdf):
    """
    Procesa un archivo PDF y extrae su texto usando OCR.
    Retorna el texto completo extraído.
    """
    try:
        # Convertir PDF a imágenes
        paginas = convert_from_path(ruta_pdf, dpi=300)
        texto_completo = ""
        
        # Procesar cada página
        for i, pagina in enumerate(paginas):
            # Aplicar OCR
            texto_pagina = pytesseract.image_to_string(pagina, lang='spa')
            texto_completo += texto_pagina + "\n"
            
        return texto_completo.strip()
    
    except Exception as e:
        return f"Error al procesar {os.path.basename(ruta_pdf)}: {str(e)}"

# Configuración
carpeta_pdfs = "errores"  # Carpeta donde están los PDFs
archivo_excel_salida = "errores.xlsx"  # Nombre del archivo Excel de salida

# Obtener lista de PDFs en la carpeta
pdfs = [f for f in os.listdir(carpeta_pdfs) if f.lower().endswith('.pdf')]

if not pdfs:
    print("❌ No se encontraron archivos PDF en la carpeta especificada.")
else:
    print(f"✅ Se encontraron {len(pdfs)} archivos PDF para procesar.")
    
    # Crear un diccionario para almacenar los resultados
    resultados = {
        'Nombre_Archivo': [],
        'Texto_Extraido': []
    }
    
    # Procesar cada PDF
    for pdf in tqdm(pdfs, desc="Procesando PDFs"):
        ruta_completa = os.path.join(carpeta_pdfs, pdf)
        
        # Extraer texto
        texto = procesar_pdf(ruta_completa)
        
        # Guardar resultados
        resultados['Nombre_Archivo'].append(pdf)
        resultados['Texto_Extraido'].append(texto)
    
    # Crear DataFrame y guardar en Excel
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel(archivo_excel_salida, index=False)
    
    print(f"\n✅ Proceso completado. Los textos han sido guardados en '{archivo_excel_salida}'")
    print(f"   Se procesaron {len(pdfs)} archivos PDF.")
    
    # Mostrar las primeras filas del DataFrame
    print("\nPrimeras entradas del Excel generado:")
    print(df_resultados.head())

✅ Se encontraron 10 archivos PDF para procesar.


Procesando PDFs: 100%|██████████| 10/10 [08:57<00:00, 53.74s/it]


✅ Proceso completado. Los textos han sido guardados en 'errores.xlsx'
   Se procesaron 10 archivos PDF.

Primeras entradas del Excel generado:
                               Nombre_Archivo  \
0                              archivo_22.pdf   
1                 Ordenanzas_Dimao_fila_3.pdf   
2            Ordenanzas_Inspección_fila_7.pdf   
3  Ordenanzas_Municipales_Sin_Modi_fila_3.pdf   
4  Ordenanzas_Municipales_Sin_Modi_fila_4.pdf   

                                      Texto_Extraido  
0  REPUBLICA DE CHILE Decreto N*. 90 18 3'5 '0 4 ...  
1  REPÚBLICA DE CHILE DecreioN* 994 20 JUL 2021\n...  
2  SECRETARÍA MUNICIPAL\n\nDECRETO ALCALDICIO N22...  
3  Transcripción actualizada al 01-01-2021,\n\nin...  
4  REPUBLICA DE CHILE — |. MUNICIPALIDAD DE RENCA...  
